<a href="https://colab.research.google.com/github/karmanandan/Spacy_NER_train_model/blob/main/Spacy_NER_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

2023-05-08 04:56:58.802345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import json

with open('/content/250_sentences_annotated_data2.json','r') as file:
  data = [json.loads(line) for line in file]

In [ ]:
type(data)

list

In [ ]:
len(data)

250

In [ ]:
data[0]

{'id': 3062,
 'text': "Summary\n\nSummary Companies Tesla down as Q3 deliveries miss market estimates\n\nU.S. factory activity slowest in ~2.5 years in Sept -ISM\n\nCredit Suisse, Citi cut 2022 year-end target for S&P 500\n\nIndexes up: Dow 2.66%, S&P 500 2.59%, Nasdaq 2.27%\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOct 3 (Reuters) - Wall Street's three major indexes rallied to close over 2% on Monday as U.S. Treasury yields tumbled on weaker-than-expected manufacturing data, increasing the appeal of stocks at the start of the year's final quarter.",
 'label': [[27, 33, 'CUSTOM_ORG'],
  [78, 81, 'CUSTOM_PLACE'],
  [136, 149, 'CUSTOM_ORG'],
  [151, 156, 'CUSTOM_ORG'],
  [267, 274, 'CUSTOM_ORG'],
  [350, 353, 'CUSTOM_PLACE']]}

In [ ]:
training_data = []
for row in data:
  temp = {}
  temp['text'] = row['text']
  temp['label'] = row['label']
  training_data.append(temp) 

In [ ]:
training_data[5]

{'text': 'Oil majors Exxon Mobil Corp (XOM.N) and Chevron Corp rose more than 5%, tracking a jump in crude prices as sources said the Organization of the Petroleum Exporting Countries and its allies are considering their biggest output cut since the start of the COVID-19 pandemic.',
 'label': [[11, 27, 'CUSTOM_ORG'], [40, 52, 'CUSTOM_ORG']]}

In [ ]:
# convert json format to Spacy format

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
blank_model = spacy.blank('en')

In [ ]:
doc_bin = DocBin()

In [ ]:
from spacy.util import filter_spans

In [ ]:
for sample in tqdm(training_data):
  text = sample['text']
  labels = sample['label']

  doc = blank_model.make_doc(text)
  ents = []
  for start, end, label in labels:
    span = doc.char_span(start, end, label=label, alignment_mode='contract')
    if span is None:
      print('skip')
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  doc_bin.add(doc)

 97%|█████████▋| 243/250 [00:00<00:00, 1235.22it/s]

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip

100%|██████████| 250/250 [00:00<00:00, 1177.71it/s]

In [ ]:
doc_bin.to_disk('train.spacy')

In [ ]:
doc_bin

In [ ]:
#Basic config generated from spacy website https://spacy.io/usage/training#quickstart
#Selected ner on UI
!python -m spacy init fill-config base_config.cfg config.cfg

2023-05-08 05:23:28.267625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
#start training
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

2023-05-08 05:24:25.204554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-05-08 05:24:28,672] [INFO] Set up nlp object from config
[2023-05-08 05:24:28,697] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-05-08 05:24:28,702] [INFO] Created vocabulary
[2023-05-08 05:24:32,241] [INFO] Added vectors: en_core_web_lg
[2023-05-08 05:24:36,516] [INFO] Finished initializing nlp object
[2023-05-08 05:24:37,972] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.56    0.00    0.00    0.00    0.

In [ ]:
# Inference Code
nlp_ner = spacy.load("/content/model-best")

# doc = nlp_ner("Joerg Steinbach, the regional economy minister of Brandenburg, where Tesla has its factory near Berlin") 
doc = nlp_ner("Oil majors Exxon Mobil Corp (XOM.N) and Chevron Corp rose more than 5%, tracking a jump in crude prices as sources said the Organization of the Petroleum Exporting Countries and its allies are considering their biggest output cut since the start of the COVID-19 pandemic.")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Exxon Mobil Corp 11 27 CUSTOM_ORG
Chevron Corp 40 52 CUSTOM_ORG


In [ ]:
spacy.displacy.render(doc, style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Joerg Steinbach\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">CUSTOM_PERSON</span>\n</mark>\n, the \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    regional economy minister\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">CUSTOM_ROLE</span>\n</mark>\n of \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Brandenburg\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left

In [ ]:
nlp_ner.get_pipe("ner").labels

('CUSTOM_ORG', 'CUSTOM_PERSON', 'CUSTOM_PLACE', 'CUSTOM_ROLE')